<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/annotation_avec_LLM/gpt4_1_guide_segment%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

# =========================
# 0. CLIENT
# =========================

client = OpenAI(api_key="")

# =========================
# 1. МОДЕЛЬ
# =========================

MODEL_NAME = "gpt-4.1"

# =========================
# 2. PROMPTS DES TESTS
# =========================

CRAN_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST CRAN — mot exclusivement figé
Question : l’expression contient-elle un mot exclusivement figé ?

Un mot exclusivement figé :
- n’existe pas comme mot autonome,
- n’apparaît jamais en dehors de l’expression,
- n’a plus d’usage productif moderne.

Exemple OUI : « us » dans « us et coutumes »
Exemple NON : « bleu » dans « cordon bleu »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

MORPH_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST MORPH — modification morphologique
Question : la flexion (pluriel, genre) rend-elle l’expression agrammaticale ou étrange ?

Exemple OUI : « garde du corps » → « gardes du corps »
Exemple NON : « livre scolaire » → « livres scolaires »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

SYNT_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST SYNT — modification syntaxique
Question : un changement d’ordre ou de structure modifie-t-il fortement le sens ou le rend-il agrammaticale ou bizarre ?

Exemple OUI : « pomme de terre » ≠ « pomme terrestre »
Exemple NON : « comité régional » ≈ « comité de la région »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

LEX_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST LEX — substitution lexicale
Question : remplacer un élément par un synonyme rend-il l’expression anormale, agrammaticale ou bizarre ?

Exemple OUI : « eau de vie » → « boisson de vie »
Exemple NON : « prix réduit » → « tarif réduit »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

MODIF_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST MODIF — insertion d’un modifieur
Question : l’insertion d’un modifieur interne rend-elle l’expression anormale ou agrammaticale ?

Exemple OUI : « pomme verte de terre »
Exemple NON : « roman contemporain policier »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

SEMREST_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST SEM-REST — restrictions sémantiques

Question : la tête nominale telle qu’elle est utilisée
a-t-elle un ensemble restreint de collocatifs naturels ?

⚠️ Analyse UNIQUEMENT la tête nominale.

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.

Expression : {expression}
Contexte : {contexte}
"""

ID_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST ID — type naturel

Question : l’expression désigne-t-elle une sous-catégorie
naturelle et littérale du noyau nominal ?

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.

Expression : {expression}
Contexte : {contexte}
"""

OPAC_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST OPAC — opacité sémantique

Question : le sens global ne correspond-il pas
à la somme des sens littéraux ?

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.

Expression : {expression}
Contexte : {contexte}
"""

# =========================
# 3. APPEL D’UN TEST
# =========================

def ask_test(prompt_template, expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)

    prompt = prompt_template.format(
        expression=expression,
        contexte=contexte
    )

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=40,
    )

    full = (resp.choices[0].message.content or "").strip()
    first_line = full.split("\n")[0].strip().upper()
    decision = "OUI" if first_line == "OUI" else "NON"

    explanation = ""
    if "\n" in full:
        explanation = full.split("\n", 1)[1].strip()

    return decision, explanation

# =========================
# 4. ARBRE DE DÉCISION
# =========================

def annotate_expression(expression, examples):
    tests = {}

    d, e = ask_test(CRAN_PROMPT, expression, examples)
    tests["CRAN"] = (d, e)
    if d == "OUI":
        return tests, "expression_idiomatique"

    d, e = ask_test(MORPH_PROMPT, expression, examples)
    tests["MORPH"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(SYNT_PROMPT, expression, examples)
    tests["SYNT"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(LEX_PROMPT, expression, examples)
    tests["LEX"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(MODIF_PROMPT, expression, examples)
    tests["MODIF"] = (d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        return tests, "collocation_opaque" if d3 == "OUI" else "collocation_transparente"

    d, e = ask_test(SEMREST_PROMPT, expression, examples)
    tests["SEM-REST"] = (d, e)
    if d == "NON":
        return tests, "expression_libre"

    d, e = ask_test(ID_PROMPT, expression, examples)
    tests["ID"] = (d, e)
    if d == "NON":
        return tests, "expression_idiomatique"

    d, e = ask_test(OPAC_PROMPT, expression, examples)
    tests["OPAC"] = (d, e)

    return tests, "collocation_opaque" if d == "OUI" else "collocation_transparente"

# =========================
# 5. FORMAT SORTIE
# =========================

def generate_annotation_with_llm(expression, examples):
    tests, category = annotate_expression(expression, examples)

    lines = []
    lines.append("Votes :")
    lines.append(f"- {category}")
    lines.append(f"\nCATÉGORIE FINALE : {category}")
    lines.append("\nExplications des tests :")

    for t in ["CRAN","MORPH","SYNT","LEX","MODIF","SEM-REST","ID","OPAC"]:
        if t in tests:
            d, e = tests[t]
            lines.append(f"{t} : {d}")
            if e:
                lines.append(f"  Explication : {e}")

    return "\n".join(lines)

# =========================
# 6. TRAITEMENT EXCEL
# =========================

input_file = "nominal_part_7.xlsx"
df = pd.read_excel(input_file)
df["llm_raw_response"] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Annotation EP"):
    expr = row.get("expression")
    ex = row.get("examples_joined")

    if pd.isna(expr) or not str(expr).strip():
        df.at[idx, "llm_raw_response"] = "N/A"
        continue

    df.at[idx, "llm_raw_response"] = generate_annotation_with_llm(expr, ex)

output_file = "annotated_nominal_part_7_gpt41_nofewshot.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved: {output_file}")
